In [1]:
## DATA MANIPULATION
import numpy as np
import pickle

## TF-IDF VECTORIZER
from sklearn.feature_extraction.text import TfidfVectorizer

## CLASSIFICATION
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

### LOAD PREPROCESSED DATA

In [2]:
f = open('../data/preprocessed.pkl','rb')
train, valid = pickle.load(f)

### CREATE BAG OF WORDS (TF-IDF) TRANSFORMER

In [3]:
tfidf = TfidfVectorizer(ngram_range=(1,2),
                        min_df=3, 
                        max_df=0.9, 
                        strip_accents='unicode', 
                        use_idf=1,
                        smooth_idf=1, 
                        sublinear_tf=1)

X_train = tfidf.fit_transform(train['comment_text'])      # vectorized train x
X_valid = tfidf.transform(valid['comment_text'])          # vectorized valid x

### STRONG BASELINE (LINEAR MODEL): NB-SVM
- based on Wang & Manning, 2012 (https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf)
- A SVM model (liblinear) that takes transformed features using Naive Bayes' log-count ratio

In [4]:
## NB-SVM MODEL
def NB_SVM(x,y):
    y = y.values
    sum_1 = x[y==1].sum(axis=0)+1           # Feature Sum for Class 1
    p_1 = (sum_1) / ((y==1).sum())          # Convert to ratio of feature in class 1 - p(f|1)

    sum_0 = x[y==0].sum(axis=0)+1           # Feature Sum for Class 0
    p_0 = (sum_0) / ((y==0).sum())          # Convert to ratio of feature in class 0 - p(f|0) 

    r = np.log(p_1/p_0)                     # Compute log ratios (the transformation matrix)
    x_nb = x.multiply(r)                    # Obtain NB feature
    
    m = LinearSVC(class_weight='balanced')  # Balanced class weights [n_samples / n_classes * np.bincount(y)]
    m.fit(x_nb,y)                           # Fit model
    return m , r                            # return fitted model & transformation matrix (need for X_valid / X_test)

labels = train.columns[2:]                  # Grab labels

## CREATE A MODEL FOR EACH CATEGORY
for label in labels:
    # Get model and transformation matrix for category
    m,r = NB_SVM(X_train, train[label])
    # Get predictions
    preds = m.predict(X_valid.multiply(r))
    # Evaluate predictions
    print('Results for {0} comments: Accuracy - {1:.2f}; Precision - {2:.2f}; Recall - {3:.2f}; F1 - {4:.2f}'.format(
                                    label, 
                                    accuracy_score(valid[label], preds), 
                                    precision_score(valid[label], preds), 
                                    recall_score(valid[label], preds),
                                    f1_score(valid[label], preds)))

Results for toxic comments: Accuracy - 0.96; Precision - 0.83; Recall - 0.75; F1 - 0.79
Results for severe_toxic comments: Accuracy - 0.99; Precision - 0.39; Recall - 0.34; F1 - 0.37
Results for obscene comments: Accuracy - 0.98; Precision - 0.84; Recall - 0.76; F1 - 0.80
Results for threat comments: Accuracy - 1.00; Precision - 0.68; Recall - 0.38; F1 - 0.48
Results for insult comments: Accuracy - 0.97; Precision - 0.75; Recall - 0.65; F1 - 0.70
Results for identity_hate comments: Accuracy - 0.99; Precision - 0.56; Recall - 0.32; F1 - 0.40


### LOGISTIC REGRESSION

In [5]:
for label in labels:
    # Create & Fit model
    m = LogisticRegression(solver='saga',class_weight='balanced')
    m.fit(X_train, train[label])
    # Get predictions
    preds = m.predict(X_valid)
    # Evaluate predictions
    print('Results for {0} comments: Accuracy - {1:.2f}; Precision - {2:.2f}; Recall - {3:.2f}; F1 - {4:.2f}'.format(
                                    label, 
                                    accuracy_score(valid[label], preds), 
                                    precision_score(valid[label], preds), 
                                    recall_score(valid[label], preds),
                                    f1_score(valid[label], preds)))

Results for toxic comments: Accuracy - 0.95; Precision - 0.67; Recall - 0.86; F1 - 0.75
Results for severe_toxic comments: Accuracy - 0.93; Precision - 0.12; Recall - 0.98; F1 - 0.22
Results for obscene comments: Accuracy - 0.98; Precision - 0.75; Recall - 0.87; F1 - 0.81
Results for threat comments: Accuracy - 0.82; Precision - 0.02; Recall - 0.96; F1 - 0.03
Results for insult comments: Accuracy - 0.89; Precision - 0.32; Recall - 0.96; F1 - 0.48
Results for identity_hate comments: Accuracy - 0.42; Precision - 0.02; Recall - 1.00; F1 - 0.03


### SVM

In [6]:
for label in labels:
    # Create & Fit model
    m = LinearSVC(class_weight='balanced')
    m.fit(X_train, train[label])
    # Get predictions
    preds = m.predict(X_valid)
    # Evaluate predictions
    print('Results for {0} comments: Accuracy - {1:.2f}; Precision - {2:.2f}; Recall - {3:.2f}; F1 - {4:.2f}'.format(
                                    label, 
                                    accuracy_score(valid[label], preds), 
                                    precision_score(valid[label], preds), 
                                    recall_score(valid[label], preds),
                                    f1_score(valid[label], preds)))

Results for toxic comments: Accuracy - 0.96; Precision - 0.79; Recall - 0.78; F1 - 0.79
Results for severe_toxic comments: Accuracy - 0.99; Precision - 0.39; Recall - 0.52; F1 - 0.45
Results for obscene comments: Accuracy - 0.98; Precision - 0.82; Recall - 0.81; F1 - 0.82
Results for threat comments: Accuracy - 1.00; Precision - 0.58; Recall - 0.47; F1 - 0.52
Results for insult comments: Accuracy - 0.97; Precision - 0.71; Recall - 0.75; F1 - 0.73
Results for identity_hate comments: Accuracy - 0.99; Precision - 0.51; Recall - 0.47; F1 - 0.49


### XGBoost

In [7]:
for label in labels:
    # Create & Fit model
    m = XGBClassifier(n_estimators=100,
                      scale_pos_weight= sum(train[label]==0) / sum(train[label]==1),
                      n_jobs=-1)
    m.fit(X_train, train[label])
    # Get predictions
    preds = m.predict(X_valid)
    # Evaluate predictions
    print('Results for {0} comments: Accuracy - {1:.2f}; Precision - {2:.2f}; Recall - {3:.2f}; F1 - {4:.2f}'.format(
                                    label, 
                                    accuracy_score(valid[label], preds), 
                                    precision_score(valid[label], preds), 
                                    recall_score(valid[label], preds),
                                    f1_score(valid[label], preds)))

Results for toxic comments: Accuracy - 0.94; Precision - 0.63; Recall - 0.82; F1 - 0.71
Results for severe_toxic comments: Accuracy - 0.98; Precision - 0.30; Recall - 0.74; F1 - 0.43
Results for obscene comments: Accuracy - 0.98; Precision - 0.73; Recall - 0.88; F1 - 0.80
Results for threat comments: Accuracy - 1.00; Precision - 0.33; Recall - 0.52; F1 - 0.41
Results for insult comments: Accuracy - 0.96; Precision - 0.54; Recall - 0.84; F1 - 0.66
Results for identity_hate comments: Accuracy - 0.98; Precision - 0.25; Recall - 0.65; F1 - 0.36
